# Multi-region data-constrained recurrent neural network (RNN) models and perform current-based decomposition (CURBD)

We are now ready to implement the CURBD from scratch.
In particular, we will train this RNN to function as a surrogate brain.
See the orignial code at: https://github.com/rajanlab/CURBD/blob/master/curbd.py

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset


### Here is a list of parameters in the original codes    
    activity: numpy.array
        N X T
    dtData: float
        time step (in s) of the training data
    dtFactor: float
        number of interpolation steps for RNN
    g: float
        instability (chaos); g<1=damped, g>1=chaotic
    tauRNN: float
        decay constant of RNN units
    tauWN: float
        decay constant on filtered white noise inputs
    ampInWN: float
        input amplitude of filtered white noise
    nRunTrain: int
        number of training runs
    nRunFree: int
        number of untrained runs at end
    P0: float
        learning rate
    nonLinearity: function
        inline function for nonLinearity
    resetPoints: list of int
        list of indeces into T. default to only set initial state at time 1.
    plotStatus: bool
        whether to plot data fits during training
    verbose: bool
        whether to print status updates
    regions: dict()
        keys are region names, values are np.array of indeces.

In [2]:
g = 0.1 #instability (chaos); g<1=damped, g>1=chaotic
num_neurons = 20 #Should based on data
num_time = 22 #Should based on data

We use some beta data for development. 

In [3]:
train = np.random.rand(num_neurons,num_time)

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, time_dim):
        super(RNNModel, self).__init__()
        
        self.hidden_dim = hidden_dim  # Number of hidden dimensions
        self.layer_dim = layer_dim    # Number of hidden layers, here the default is one
        
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')  # RNN
        self.fc = nn.Linear(hidden_dim, output_dim)  # Here use a linear layer as Readout layer
        
        # Initialize hidden-to-hidden weight to random values
        for l in range(layer_dim):
            # Get the attribute name for the hidden-to-hidden weight for the layer 'l'
            weight_name = f"weight_hh_l{l}"
            getattr(self.rnn, weight_name).data.uniform_(-g, g)

    def forward(self, x, k=1):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        
        outputs = []
        for _ in range(time_dim):
            # Perform one step of the RNN
            out, h0 = self.rnn(x, h0)
            # Process the RNN output through the FC layer
            out = self.fc(out[:, -1, :])
            outputs.append(out)
            # Use the current output as the next input (May need to reshape it properly)
            x = out.unsqueeze(1)
        
        # Stack outputs to have them in [batch, k, output_dim] shape
        return torch.stack(outputs, dim=1)


# batch_size, epoch and iteration
#batch_size = 100
#n_iters = 8000
#num_epochs = n_iters / (len(features_train) / batch_size)
#num_epochs = int(num_epochs)

# Pytorch train and test sets
#train = TensorDataset(featuresTrain,targetsTrain)
#test = TensorDataset(featuresTest,targetsTest)

# data loader
#train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
#test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    

In [5]:


# Generate multiple sine waves with different frequencies as an example time series
time_steps = np.linspace(0, 10, 1000)
data = np.array([np.sin(time_steps * (i * 0.1 + 1)) for i in range(20)]).T

### Data Preparation
For simplicity, let's assume each input sequence to the RNN consists of 10 values, and we want to predict the next 10 values.

In [7]:
seq_length = num_time
train_data = []
train_labels = []

for i in range(len(data) - seq_length*2):
    train_data.append(data[i:i+seq_length])
    train_labels.append(data[i+seq_length:i+2*seq_length])

# Convert lists of numpy arrays to numpy arrays
train_data_np = np.array(train_data)
train_labels_np = np.array(train_labels)

# Convert numpy arrays to tensors
train_data = torch.tensor(train_data_np, dtype=torch.float32)
train_labels = torch.tensor(train_labels_np, dtype=torch.float32)


### Training Loop

In [8]:
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

# Hyperparameters
input_dim = num_neurons
hidden_dim = 32
layer_dim = 1
output_dim = num_neurons
learning_rate = 0.001
epochs = 200
time_dim = num_time 

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim, time_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(train_data)
    loss = criterion(outputs, train_labels)
    
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [1/200], Loss: 0.5275
Epoch [11/200], Loss: 0.5135
Epoch [21/200], Loss: 0.5036
Epoch [31/200], Loss: 0.4756
Epoch [41/200], Loss: 0.4510
Epoch [51/200], Loss: 0.4264
Epoch [61/200], Loss: 0.3828
Epoch [71/200], Loss: 0.3480
Epoch [81/200], Loss: 0.3193
Epoch [91/200], Loss: 0.2870
Epoch [101/200], Loss: 0.2493
Epoch [111/200], Loss: 0.2204
Epoch [121/200], Loss: 0.1984
Epoch [131/200], Loss: 0.1802
Epoch [141/200], Loss: 0.1650
Epoch [151/200], Loss: 0.1451
Epoch [161/200], Loss: 0.1303
Epoch [171/200], Loss: 0.1186
Epoch [181/200], Loss: 0.1084
Epoch [191/200], Loss: 0.0976
